In [ ]:
#Inspiration https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/

In [11]:
#Download Movie Review dataset
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/review_polarity.tar.gz

--2022-01-20 09:21:47--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/review_polarity.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/octet-stream]
Saving to: ‘review_polarity.tar.gz.1’

review_polarity.tar 100%[===================>]   2.98M  --.-KB/s    in 0.07s   

2022-01-20 09:21:47 (44.9 MB/s) - ‘review_polarity.tar.gz.1’ saved [3127238/3127238]



In [12]:
#Unzip tar.gz file
!tar -xf /content/review_polarity.tar.gz

In [13]:
#Load Libraries
from string import punctuation
from os import listdir
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [26]:
#Helper Functions for Vocab Prepration
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
	# load doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# update counts
	vocab.update(tokens)

# load all docs in a directory
def process_docs(directory, vocab, is_trian):
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_trian and filename.startswith('cv9'):
			continue
		if not is_trian and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# add doc to vocab
		add_doc_to_vocab(path, vocab)



In [15]:
#!pip install nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs('txt_sentoken/neg', vocab, True)
process_docs('txt_sentoken/pos', vocab, True)
# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

44276
[('film', 7983), ('one', 4946), ('movie', 4826), ('like', 3201), ('even', 2262), ('good', 2080), ('time', 2041), ('story', 1907), ('films', 1873), ('would', 1844), ('much', 1824), ('also', 1757), ('characters', 1735), ('get', 1724), ('character', 1703), ('two', 1643), ('first', 1588), ('see', 1557), ('way', 1515), ('well', 1511), ('make', 1418), ('really', 1407), ('little', 1351), ('life', 1334), ('plot', 1288), ('people', 1269), ('bad', 1248), ('could', 1248), ('scene', 1241), ('movies', 1238), ('never', 1201), ('best', 1179), ('new', 1140), ('scenes', 1135), ('man', 1131), ('many', 1130), ('doesnt', 1118), ('know', 1092), ('dont', 1086), ('hes', 1024), ('great', 1014), ('another', 992), ('action', 985), ('love', 977), ('us', 967), ('go', 952), ('director', 948), ('end', 946), ('something', 945), ('still', 936)]


In [ ]:
vocab.most_common(50)

In [28]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

25767


In [29]:
#Save Vocab

def save_list(lines, filename):
	# convert lines to a single blob of text
	data = '\n'.join(lines)
	# open file
	file = open(filename, 'w')
	# write text
	file.write(data)
	# close file
	file.close()
 
# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')

In [ ]:
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

#Train Embedding Layer

In [32]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

In [33]:
# turn a doc into clean tokens
def clean_doc(doc, vocab):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# filter out tokens not in vocab
	tokens = [w for w in tokens if w in vocab]
	tokens = ' '.join(tokens)
	return tokens

In [34]:
# load all docs in a directory
def process_docs(directory, vocab, is_train):
	documents = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_train and filename.startswith('cv9'):
			continue
		if not is_train and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load the doc
		doc = load_doc(path)
		# clean doc
		tokens = clean_doc(doc, vocab)
		# add to list
		documents.append(tokens)
	return documents
 
# load all training reviews
positive_docs = process_docs('txt_sentoken/pos', vocab, True)
negative_docs = process_docs('txt_sentoken/neg', vocab, True)
train_docs = negative_docs + positive_docs

In [35]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_docs)

In [45]:
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)

In [46]:
# pad sequences
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [47]:
# define training labels
ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])

In [48]:
# load all TEST reviews
positive_docs = process_docs('txt_sentoken/pos', vocab, False)
negative_docs = process_docs('txt_sentoken/neg', vocab, False)
test_docs = negative_docs + positive_docs
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(test_docs)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define test labels
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])

In [49]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

In [50]:
vocab_size, max_length

(25768, 1317)

In [51]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1317, 100)         2576800   
                                                                 
 conv1d_1 (Conv1D)           (None, 1310, 32)          25632     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 655, 32)          0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 20960)             0         
                                                                 
 dense_2 (Dense)             (None, 10)                209610    
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                      

In [52]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)

Epoch 1/10
57/57 - 18s - loss: 0.6913 - accuracy: 0.5183 - 18s/epoch - 323ms/step
Epoch 2/10
57/57 - 17s - loss: 0.6017 - accuracy: 0.6939 - 17s/epoch - 301ms/step
Epoch 3/10
57/57 - 17s - loss: 0.1865 - accuracy: 0.9428 - 17s/epoch - 295ms/step
Epoch 4/10
57/57 - 18s - loss: 0.0170 - accuracy: 0.9983 - 18s/epoch - 324ms/step
Epoch 5/10
57/57 - 19s - loss: 0.0037 - accuracy: 1.0000 - 19s/epoch - 333ms/step
Epoch 6/10
57/57 - 19s - loss: 0.0019 - accuracy: 1.0000 - 19s/epoch - 326ms/step
Epoch 7/10
57/57 - 17s - loss: 0.0013 - accuracy: 1.0000 - 17s/epoch - 303ms/step
Epoch 8/10
57/57 - 17s - loss: 9.7960e-04 - accuracy: 1.0000 - 17s/epoch - 302ms/step
Epoch 9/10
57/57 - 19s - loss: 7.8532e-04 - accuracy: 1.0000 - 19s/epoch - 325ms/step
Epoch 10/10
57/57 - 20s - loss: 6.5536e-04 - accuracy: 1.0000 - 20s/epoch - 343ms/step


In [53]:
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 89.499998


In [54]:
predictions=model.predict(Xtest)

In [56]:
n=89
predictions[n],'\\',test_docs[n]

(array([0.110246], dtype=float32),
 '\\',
 'ever feel youre spending whole life net ouch eating breathing web sites meaningful relationships formed net get best sex net firsttime director hal could shoot entire movie characters computers would settles characters talking phone denise calls movie electronic generation characters caught work insecurities prefer live relationships fantasies phone satire sometimes funny one let answering machines run lives problem denise calls movie idea darn good one still idea despite attempt plots subplots despite genuinely funny moments predict movies outcome within first fifteen minutes get drift series shots characters explaining phone couldnt make party nobody going meeting anybody film would rather talking phone heres plot characters dysfunctional limbo loud quirky stranger denise calls martin announce pregnant child courtesy sperm local bank martin progresses slamming phone long phone conversations babys name friends friends friends get involved co

In [57]:
mysent = tokenizer.texts_to_sequences(['director luis mandokis last film superb serious drama man loves woman luck ultimately run latest picture message \
bottle worst type romance movie tugs relentlessly violently heartstrings miraculously manages dry eyes rather tear everything occurs \
telegraphed way advance since type story done many times much theres absence suspense film ultimately moves deliberate pace trying make great \
meaningful statements plot developments becomes tedious bore sit message bottle begins theresa osborne robin wright penn single mother researcher \
chicago tribune finds bottle washed shore jogging one day inside bottle anonymous love letter addressed mystery woman named catherine \
theresa taken aback honesty sweetness shows around work objection finds editor placed letter newspaper soon heavy research conducted find \
wrote letter based type bottle ship logo top typed message traced man named garrett blake theresa finds traveling outer banks boating town \
north carolina find specifics message course garrett turns handsome rugged man around theresas age played kevin costner immediately charmed \
hesitant unveil truth finds catherine garretts late wife died years earlier think know headed likely wouldnt call giving away anything say \
pictures end movie fallen deep artificial melodrama didnt buy second positive things say message bottle performances robin wright penn \
paul newman garretts stubborn loving father far par wasteful shaggy dog love story cinematography caleb deschanel takes great advantage \
beautiful eastern coast paints chicago equally alluring city meanwhile costner yet redeem lessthanstellar films made recently seems bad \
luck wouldnt want make another movie set near water one main ridiculous set sea actors sadly wasted including illeana douglas underused \
actress seems always get stuck friend roles playing penns coworker tribune first half message bottle plays like humdrum trite television \
movie lifetime channel theresa spends great deal time getting know garrett dialogue least bit stimulating entertaining usually type person \
practically sequences since film doubt trying develop characters relationships rang resounding since dialogue felt written people really \
talking main characters film little interest say particularly interesting know immediately trouble swear watching message bottle felt read \
screenplay entirety arriving theater heck actuality im even familiar novel based nicholas sparks always one step ahead characters movie ran \
closely tightly constraints triedandtrue hollywood melodrama film strip often seemed almost danger tearing nobody wins prizes guessing garrett \
eventually find theresas secret several obstacles come within ways living happily ever exact problem occurred last years similar watch realize \
similar mean meg cage romantic drama city angels although theresa deeply touched heartfelt letter finds bottle perhaps might better finding message \
bottle preferably filming began read memo screenwriters use brain'])
mysent_pad = pad_sequences(mysent, maxlen=max_length, padding='post')
model.predict(mysent_pad)

array([[0.21889892]], dtype=float32)

In [60]:
mysent = tokenizer.texts_to_sequences(['the moview was cold'])
mysent_pad = pad_sequences(mysent, maxlen=max_length, padding='post')
model.predict(mysent_pad)

array([[0.48061985]], dtype=float32)

#Train word2vec Embedding

In [61]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

In [62]:
# turn a doc into clean tokens
def doc_to_clean_lines(doc, vocab):
	clean_lines = list()
	lines = doc.splitlines()
	for line in lines:
		# split into tokens by white space
		tokens = line.split()
		# remove punctuation from each token
		table = str.maketrans('', '', punctuation)
		tokens = [w.translate(table) for w in tokens]
		# filter out tokens not in vocab
		tokens = [w for w in tokens if w in vocab]
		clean_lines.append(tokens)
	return clean_lines

In [63]:
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
	lines = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_trian and filename.startswith('cv9'):
			continue
		if not is_trian and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load and clean the doc
		doc = load_doc(path)
		doc_lines = doc_to_clean_lines(doc, vocab)
		# add lines to list
		lines += doc_lines
	return lines

In [64]:
# load training data
positive_docs = process_docs('txt_sentoken/pos', vocab, True)
negative_docs = process_docs('txt_sentoken/neg', vocab, True)
sentences = negative_docs + positive_docs
print('Total training sentences: %d' % len(sentences))

Total training sentences: 58109


In [65]:
from string import punctuation
from os import listdir
from gensim.models import Word2Vec

# train word2vec model
model = Word2Vec(sentences, size=100, window=5, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 25767


In [66]:
# save model in ASCII (word2vec) format
filename = 'embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

#Use Pre-trained Embedding

In [67]:
# load embedding as a dict
def load_embedding(filename):
	# load embedding into memory, skip first line
	file = open(filename,'r')
	lines = file.readlines()[1:]
	file.close()
	# create a map of words to vectors
	embedding = dict()
	for line in lines:
		parts = line.split()
		# key is string word, value is numpy array for vector
		embedding[parts[0]] = asarray(parts[1:], dtype='float32')
	return embedding

In [68]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
	# total vocabulary size plus 0 for unknown words
	vocab_size = len(vocab) + 1
	# define weight matrix dimensions with all 0
	weight_matrix = zeros((vocab_size, 100))
	# step vocab, store vectors using the Tokenizer's integer mapping
	for word, i in vocab.items():
		weight_matrix[i] = embedding.get(word)
	return weight_matrix

In [70]:
# load embedding from file
raw_embedding = load_embedding('embedding_word2vec.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

In [71]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1317, 100)         2576800   
                                                                 
 conv1d_2 (Conv1D)           (None, 1313, 128)         64128     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 656, 128)         0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 83968)             0         
                                                                 
 dense_4 (Dense)             (None, 1)                 83969     
                                                                 
Total params: 2,724,897
Trainable params: 148,097
Non-trainable params: 2,576,800
______________________________________

In [72]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Epoch 1/10
57/57 - 17s - loss: 0.6979 - accuracy: 0.5433 - 17s/epoch - 301ms/step
Epoch 2/10
57/57 - 16s - loss: 0.6470 - accuracy: 0.6278 - 16s/epoch - 289ms/step
Epoch 3/10
57/57 - 17s - loss: 0.5602 - accuracy: 0.7128 - 17s/epoch - 292ms/step
Epoch 4/10
57/57 - 17s - loss: 0.4322 - accuracy: 0.8133 - 17s/epoch - 292ms/step
Epoch 5/10
57/57 - 17s - loss: 0.3160 - accuracy: 0.8772 - 17s/epoch - 291ms/step
Epoch 6/10
57/57 - 17s - loss: 0.2113 - accuracy: 0.9394 - 17s/epoch - 290ms/step
Epoch 7/10
57/57 - 17s - loss: 0.1392 - accuracy: 0.9689 - 17s/epoch - 292ms/step
Epoch 8/10
57/57 - 16s - loss: 0.0917 - accuracy: 0.9894 - 16s/epoch - 287ms/step
Epoch 9/10
57/57 - 17s - loss: 0.0561 - accuracy: 0.9944 - 17s/epoch - 294ms/step
Epoch 10/10
57/57 - 17s - loss: 0.0335 - accuracy: 0.9994 - 17s/epoch - 295ms/step
Test Accuracy: 59.500003


#pre-trained GloVe vectors from the Stanford webpage

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2022-01-19 10:19:07--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-01-19 10:19:08--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.19MB/s    in 2m 40s  

2022-01-19 10:21:48 (5.13 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
# load embedding as a dict
def load_embedding(filename):
	# load embedding into memory, skip first line
	file = open(filename,'r')
	lines = file.readlines()
	file.close()
	# create a map of words to vectors
	embedding = dict()
	for line in lines:
		parts = line.split()
		# key is string word, value is numpy array for vector
		embedding[parts[0]] = asarray(parts[1:], dtype='float32')
	return embedding

In [ ]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
	# total vocabulary size plus 0 for unknown words
	vocab_size = len(vocab) + 1
	# define weight matrix dimensions with all 0
	weight_matrix = zeros((vocab_size, 100))
	# step vocab, store vectors using the Tokenizer's integer mapping
	for word, i in vocab.items():
		vector = embedding.get(word)
		if vector is not None:
			weight_matrix[i] = vector
	return weight_matrix

In [ ]:
# load embedding from file
raw_embedding = load_embedding('glove.6B.100d.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

In [ ]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1380, 100)         4427700   
                                                                 
 conv1d_2 (Conv1D)           (None, 1376, 128)         64128     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 688, 128)         0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 88064)             0         
                                                                 
 dense_3 (Dense)             (None, 1)                 88065     
                                                                 
Total params: 4,579,893
Trainable params: 152,193
Non-trainable params: 4,427,700
______________________________________

In [ ]:
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 73.500001


#Prerained Word2Vec from Google

In [4]:
#Inspiration https://www.oreilly.com/library/view/deep-learning-essentials/9781785880360/a2364ad8-8e8c-4723-bf38-092e14f020f5.xhtml

In [3]:
from gensim.models import KeyedVectors
# Load pre-trained model
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/DS-Data/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [5]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133)]

In [12]:
a = model.word_vec('queen')
b = model.word_vec('king') - model.word_vec('man') + model.word_vec('women')

In [22]:
a = model.word_vec('man')
b = model.word_vec('star')

In [23]:
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(a, b)/(norm(a)*norm(b))
cos_sim

0.26227346